<a href="https://colab.research.google.com/github/dahlia52/Advanced-Statistical-Data-Analysis/blob/main/LeNet_cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.transforms as transforms
from torchvision.datasets import MNIST, CIFAR10, CIFAR100
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = './datasets/'

transform = transforms.Compose([transforms.ToTensor()]) # 이미지를 텐서로 변환

# Prepare Data
train_data = CIFAR100(root = path, train = True, transform = transform, download = True)
test_data = CIFAR100(root = path, train = False, transform = transform, download = True)

batch_size = 100

# DataLoader
train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True, num_workers = 4)
test_loader = DataLoader(dataset = test_data, batch_size = batch_size, shuffle = False, num_workers = 4)

print(train_data)
print(test_data)

input_shape = train_data[0][0].shape # (3,32,32)
output_shape = len(train_data.classes) # 100

print(input_shape,output_shape)

Files already downloaded and verified
Files already downloaded and verified
Dataset CIFAR100
    Number of datapoints: 50000
    Root location: ./datasets/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
           )
Dataset CIFAR100
    Number of datapoints: 10000
    Root location: ./datasets/
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
           )
torch.Size([3, 32, 32]) 100


In [ ]:
if torch.backends.mps.is_available():
    device = torch.device("mps:0")
elif torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu:0")

print(device)

cuda:0


In [ ]:
from torch.nn.modules.activation import LeakyReLU
class LeNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 18, kernel_size = (5,5), stride = 1, padding = 2) # MNIST 데이터는 흑백 데이터이므로 in_channels = 1
    self.pool1 = nn.AvgPool2d(kernel_size = (2,2), stride = 2, padding = 0)
    self.conv2 = nn.Conv2d(in_channels = 18, out_channels = 48, kernel_size = (5,5), stride = 1) # kernel의 가로, 세로 길이가 동일하므로 kernel_size = 5도 가능
    self.pool2 = nn.AvgPool2d(kernel_size = (2,2), stride = 2, padding = 0)
    self.flatten = nn.Flatten()
    self.fcs = nn.Sequential(
        nn.Linear(1728, 512),
        nn.LeakyReLU(),
        nn.Dropout(),
        nn.Linear(512, 256),
        nn.LeakyReLU(),
        nn.Dropout(),
        nn.Linear(256, 128),
        nn.LeakyReLU(),
        nn.Dropout(),
        nn.Linear(128, output_shape)
    )

  def forward(self, x):
    # print(x.shape) # (100,3,32,32) # batch_size = 100
    hidden = F.leaky_relu(self.conv1(x))
    # print(hidden.shape) # (100,18,32,32)
    hidden = self.pool1(hidden)
    # print(hidden.shape) # (100,18,16,16)
    hidden = F.leaky_relu(self.conv2(hidden))
    # print(hidden.shape) # (100,48,12,12)
    hidden = self.pool2(hidden)
    # print(hidden.shape) # (100,48,6,6)
    hidden = self.flatten(hidden)
    # print(hidden.shape) # (100,1728)
    output = self.fcs(hidden)
    # print(output.shape) # (100,100)
    return output

In [ ]:
model = LeNet().to(device)
loss = nn.CrossEntropyLoss(reduction = 'sum')
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3, weight_decay = 1e-3)

In [ ]:
num_epoch = 150
train_loss_list, test_loss_list = list(), list()

for i in range(num_epoch):
  # train
  model.train()

  total_loss = 0
  count = 0

  for batch_idx, (x, y) in enumerate(train_loader):
    x, y = x.to(device), y.to(device)
    y_est = model.forward(x)
    cost = loss(y_est, y)

    total_loss += cost.item()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    pred = torch.argmax(y_est, dim = -1)
    count += (pred == y).sum().item()

  acc = count / len(train_data)
  avg_loss = total_loss / len(train_data)

  train_loss_list.append(avg_loss)

  if i % 1 == 0:
        print("\nEpoch %d Train: Loss %.3f / Accuracy %.3f"%(i,avg_loss,acc))

  # test
  model.eval()

  total_loss = 0
  count = 0

  with torch.no_grad():
    for batch_idx, (x, y) in enumerate(test_loader):
      x, y = x.to(device), y.to(device)
      y_est = model.forward(x)
      cost = loss(y_est, y)

      total_loss += cost.item()

      pred = torch.argmax(y_est, dim = -1)
      count += (pred == y).sum().item()

    acc = count / len(test_data)
    avg_loss = total_loss / len(test_data)

    test_loss_list.append(avg_loss)

    if i % 1 == 0:
      print("Epoch %d Test: Loss %.3f / Accuracy %.3f"%(i,avg_loss,acc))


Epoch 0 Train: Loss 4.331 / Accuracy 0.035
Epoch 0 Test: Loss 4.031 / Accuracy 0.068

Epoch 1 Train: Loss 3.954 / Accuracy 0.078
Epoch 1 Test: Loss 3.703 / Accuracy 0.124

Epoch 2 Train: Loss 3.741 / Accuracy 0.115
Epoch 2 Test: Loss 3.511 / Accuracy 0.162

Epoch 3 Train: Loss 3.556 / Accuracy 0.145
Epoch 3 Test: Loss 3.328 / Accuracy 0.200

Epoch 4 Train: Loss 3.426 / Accuracy 0.170
Epoch 4 Test: Loss 3.237 / Accuracy 0.220

Epoch 5 Train: Loss 3.312 / Accuracy 0.191
Epoch 5 Test: Loss 3.166 / Accuracy 0.235

Epoch 6 Train: Loss 3.208 / Accuracy 0.209
Epoch 6 Test: Loss 3.050 / Accuracy 0.253

Epoch 7 Train: Loss 3.116 / Accuracy 0.228
Epoch 7 Test: Loss 2.956 / Accuracy 0.270

Epoch 8 Train: Loss 3.020 / Accuracy 0.246
Epoch 8 Test: Loss 2.937 / Accuracy 0.275

Epoch 9 Train: Loss 2.948 / Accuracy 0.260
Epoch 9 Test: Loss 2.848 / Accuracy 0.298

Epoch 10 Train: Loss 2.871 / Accuracy 0.275
Epoch 10 Test: Loss 2.816 / Accuracy 0.304

Epoch 11 Train: Loss 2.796 / Accuracy 0.284
Epoch 1

In [ ]:
num_parameter = 0

for parameter in model.parameters():
  print(parameter.shape)
  num_parameter += np.prod(parameter.size())
print(num_parameter)

torch.Size([18, 3, 5, 5])
torch.Size([18])
torch.Size([48, 18, 5, 5])
torch.Size([48])
torch.Size([512, 1728])
torch.Size([512])
torch.Size([256, 512])
torch.Size([256])
torch.Size([128, 256])
torch.Size([128])
torch.Size([100, 128])
torch.Size([100])
1085388
